In [1]:
from pyspark.sql import SparkSession

In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("task1").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/19 20:08:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
cves_df = spark.read.json("cves.json")

In [5]:
cpes_df = spark.read.json("cpes.json")

In [7]:
cve2cpe_df = spark.read.json("cve2cpe.json")

In [9]:
from pyspark.sql.functions import col, collect_list

In [12]:
# Join cve2cpe_df with cpes_df to find the matching CPES for each CVE
match_cpes_df = cve2cpe_df.join(cpes_df, 
                               (cpes_df.product_name == cve2cpe_df.product_name) &
                               (cpes_df.vendor == cve2cpe_df.vendor) &
                               (cpes_df.product_version == cve2cpe_df.product_version) &
                               ((cve2cpe_df.from_version == "") | (cpes_df.product_version > cve2cpe_df.from_version)) &
                               ((cve2cpe_df.to_version == "") | (cpes_df.product_version < cve2cpe_df.to_version)),
                               how='left').select(cpes_df["cpe"],
                                                  cpes_df["cpe_id"],
                                                  cve2cpe_df["cve"],
                                                  cpes_df["title"])

In [13]:
print((match_cpes_df.count(), len(match_cpes_df.columns)))


(11944914, 4)


In [14]:
# Group the matching CPES by CVE and collect them into an array
cve_match_cpes_df = match_cpes_df.groupBy('cve').agg(collect_list('cpe').alias('cpe'))

In [16]:
print((cve_match_cpes_df.count(), len(cve_match_cpes_df.columns)))

(197711, 2)


In [18]:
# Join cve_matching_cpes_df with cves_df to filter the matching CPES that exist in cves_df
output_df = cve_match_cpes_df.join(cves_df, 
                                   cve_match_cpes_df['cve'] == cves_df['cve'],
                                   how='inner').select(cve_match_cpes_df["cve"],
                                                       cves_df["description"],
                                                       cve_match_cpes_df["cpe"])

In [19]:
print((output_df.count(), len(output_df.columns)))

(197711, 3)


In [20]:
# Write the output to a JSON file
output_df.write.json('output1.json')